In [16]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

In [2]:
df = pd.read_csv("data/metrics.csv")
# df = pd.read_csv("data/antlr4.csv")
# df = pd.read_csv("data/discord4j.csv")
# df = pd.read_csv("data/junit4.csv")
df.head()

,Project,ID,Name,LongName,Parent,Component,Path,Line,Column,EndLine,...,Documentation Metric Rules,Documentation Rules,Error Prone Rules,Inheritance Metric Rules,Multithreading Rules,Performance Rules,Security Rules,Size Metric Rules,isDefective,Unnamed: 92
0,antlr4,L134,Tool,org.antlr.v4.Tool,L500,L103,C:\Users\domco\Desktop\DP\repos\antlr4\tool\sr...,89,1,859,...,80,0,4,0,0,18,0,21,1,NaN
1,antlr4,L125,Antlr4ErrorLog,org.antlr.mojo.antlr4.Antlr4ErrorLog,L107,L103,C:\Users\domco\Desktop\DP\repos\antlr4\antlr4-...,47,1,124,...,0,0,0,0,0,0,0,0,0,NaN
2,antlr4,L199,Antlr4Mojo,org.antlr.mojo.antlr4.Antlr4Mojo,L107,L103,C:\Users\domco\Desktop\DP\repos\antlr4\antlr4-...,77,1,505,...,26,0,2,0,0,1,0,10,0,NaN
3,antlr4,L643,ANTLRFileStream,org.antlr.v4.runtime.ANTLRFileStream,L507,L103,C:\Users\domco\Desktop\DP\repos\antlr4\runtime...,42,1,86,...,11,0,1,0,0,1,0,2,0,NaN
4,antlr4,L645,ANTLRInputStream,org.antlr.v4.runtime.ANTLRInputStream,L507,L103,C:\Users\domco\Desktop\DP\repos\antlr4\runtime...,47,1,246,...,33,2,0,0,0,1,0,7,0,NaN


## Take the working set of reatures
NOA: Number of Attributes <br>
NOP: Number of Parameters <br>
NOC: Number of Children <br>
CBO: Coupling Between Objects <br>
DIT: Depth Inheritance Tree <br>
RFC: Response for a Class <br>
LCOM5: Lack of Cohesion of Methods <br>
<i>Labels</i><br>
1, if LOC<=75; 0 otherwise <br>

In [7]:
# Data preparation
features = df[['NOA', 'NOP', 'NOC', 'CBO', 'DIT', 'RFC', 'LCOM5']]
labels = [(loc <= 75) for loc in df['LOC']]
y = np.ravel(labels) 

In [8]:
# Data standartization
FeaturScaler = StandardScaler()
 
# Storing the fit object for later reference
FeaturScalerFit = FeaturScaler.fit(features)
 
# Generating the standardized values of X and y
X = FeaturScalerFit.transform(features)

In [9]:
# Split the data into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42) 

# Quick sanity check with the shapes of Training and testing datasets
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1679, 7)
(1679,)
(720, 7)
(720,)


## Finding best hyperparameters

In [14]:
# Defining a function to find the best parameters for ANN
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list = [5, 10, 15, 20]
    epoch_list = [5, 10, 50, 100]
    
    SearchResultsData = pd.DataFrame(columns=['TrialNumber', 'Parameters', 'Accuracy'])
    
    # initializing the trials
    TrialNumber=0
    best_Accuracy = 0
    best_batch_size = 0
    best_epochs = 0
    
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            TrialNumber += 1
            
           # Creating the classifier ANN model
            classifier = Sequential()
            classifier.add(Dense(units=8, input_dim=X_train.shape[1], kernel_initializer='uniform', activation='relu'))
            classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))
            classifier.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))
            classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

            ANN_Model = classifier.fit(X_train, y_train, batch_size=batch_size_trial, epochs=epochs_trial, verbose=0)
            
            # Fetching the accuracy of the training
            Accuracy = ANN_Model.history['accuracy'][-1]

            # printing the results of the current iteration
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'Accuracy:', Accuracy)
            
            SearchResultsData = SearchResultsData.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), Accuracy]],
                                                                    columns=['TrialNumber', 'Parameters', 'Accuracy'] ))
            if TrialNumber==1 or best_Accuracy<Accuracy:
                best_Accuracy = Accuracy
                best_batch_size = batch_size_trial
                best_epochs = epochs_trial
          
    return(best_batch_size, best_epochs)
 
# Calling the function
best_batch_size, best_epochs = FunctionFindBestParams(X_train, y_train, X_test, y_test)
print(best_batch_size, best_epochs)

1 Parameters: batch_size: 5 - epochs: 5 Accuracy: 0.9434186816215515
2 Parameters: batch_size: 5 - epochs: 10 Accuracy: 0.9505658149719238
3 Parameters: batch_size: 5 - epochs: 50 Accuracy: 0.9523525834083557
4 Parameters: batch_size: 5 - epochs: 100 Accuracy: 0.9535437822341919
5 Parameters: batch_size: 10 - epochs: 5 Accuracy: 0.9428231120109558
6 Parameters: batch_size: 10 - epochs: 10 Accuracy: 0.9493746161460876
7 Parameters: batch_size: 10 - epochs: 50 Accuracy: 0.9523525834083557
8 Parameters: batch_size: 10 - epochs: 100 Accuracy: 0.9559261202812195
9 Parameters: batch_size: 15 - epochs: 5 Accuracy: 0.9404407143592834
10 Parameters: batch_size: 15 - epochs: 10 Accuracy: 0.9475878477096558
11 Parameters: batch_size: 15 - epochs: 50 Accuracy: 0.9523525834083557
12 Parameters: batch_size: 15 - epochs: 100 Accuracy: 0.9553305506706238
13 Parameters: batch_size: 20 - epochs: 5 Accuracy: 0.9309112429618835
14 Parameters: batch_size: 20 - epochs: 10 Accuracy: 0.944014310836792
15 Para

## Testing the ANN model with the best parameters

In [18]:
# Create model
classifier = Sequential()
classifier.add(Dense(units=8, input_dim=X_train.shape[1], kernel_initializer='uniform', activation='relu'))
classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))
classifier.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = best_batch_size, epochs = best_epochs, verbose=0)
 
# Generating Predictions on testing data
Predictions = classifier.predict(X_test)
 
# Scaling the test data back to original scale
Test_Data = FeaturScalerFit.inverse_transform(X_test)

23/23 [==============================] - 0s 810us/step


In [21]:
# Generating a data frame for analyzing the test data
#TestingData = pd.DataFrame(data=Test_Data, columns=features)
TestingData = pd.DataFrame()
TestingData['LOC class'] = y_test
TestingData['PredictedLOC-Probab'] = Predictions
 
# Defining the probability threshold
def probThreshold(inpProb):
    if inpProb  >  0.5:
        return(1)
    else:
        return(0)

# Generating predictions on the testing data by applying probability threshold
TestingData['PredictedLOC-class'] = TestingData['PredictedLOC-Probab'].apply(probThreshold)

## Finding the accuracy of the model

In [22]:
print('\n######### Testing Accuracy Results #########')
print(metrics.classification_report(TestingData['LOC class'], TestingData['PredictedLOC-class']))
print(metrics.confusion_matrix(TestingData['LOC class'], TestingData['PredictedLOC-class']))


######### Testing Accuracy Results #########
              precision    recall  f1-score   support

       False       0.88      0.80      0.84       107
        True       0.97      0.98      0.97       613

    accuracy                           0.95       720
   macro avg       0.92      0.89      0.91       720
weighted avg       0.95      0.95      0.95       720

[[ 86  21]
 [ 12 601]]
